In [ ]:
#libraries
import cv2
import numpy as np
import pandas as pd
from scipy.spatial.distance import euclidean
from sklearn.neighbors import KDTree
import time
import argparse
import json
import xml.etree.ElementTree as ET
from tqdm import tqdm
import zipfile
from skimage.color import rgb2hsv,rgb2lab, hsv2rgb, lab2rgb
import os
import matplotlib.pyplot as plt

In [ ]:
# read the .xml-file
zip_ref = zipfile.ZipFile('CompanyMen_v1.0-split-012-Bobby_being_angry.azp')
zip_ref.extractall('/tmp')
tree = ET.parse('/tmp/content.xml')
root = tree.getroot().findall('./{http://experience.univ-lyon1.fr/advene/ns}annotations')
i=0
ts_end=[]
ts_begin=[]
for child in root[0].iter():
    if child.get('type')=='#Shot':
        i+=1
        for child2 in child:
            if child2.tag=='{http://experience.univ-lyon1.fr/advene/ns}millisecond-fragment':
                ts_end.append(round(int(child2.get('end'))/1000*25))
                ts_begin.append(round(int(child2.get('begin'))/1000*25))
#         if i==4:
#             break

In [ ]:
print('ts_begin: ',np.shape(ts_begin),ts_begin)
print('ts_end: ',np.shape(ts_end),ts_end)

In [ ]:
#read video file frame by frame, beginning and ending with a timestamp
def read_video_segments(video,start_frame,end_frame,resolution_width,target_colorspace):
    resolution_height=int(round(resolution_width * 9/16))
    resolution=(resolution_width,resolution_height)
    vid = cv2.VideoCapture(video)
    frames=[]
    vid_length=0
    with tqdm(total=end_frame-start_frame+1) as pbar: #init the progressbar,with max lenght of the given segment
        while(vid.isOpened()):
            # Capture frame-by-frame
            ret, frame = vid.read() # if ret is false, frame has no content
            if not ret:
                break
            # skip every "skip_frame"
            if vid_length>=start_frame:
                # resize the video to a different resolution
                frame=cv2.resize(frame,resolution)
                frame=np.array(frame,dtype='uint8')
                frames.append(frame) #add the individual frames to a list
                pbar.update(1) #update the progressbar
            if vid_length==end_frame:
                pbar.update(1)
                break
            vid_length+=1 #increase the vid_length counter
    vid.release()
    cv2.destroyAllWindows()
    frames=change_colorspace(frames,target_colorspace)
    return frames[:-1]

In [ ]:
def change_colorspace(frame_list,target_colorspace):
    changed_frame_list=[]
    if target_colorspace=='HSV':
        print('HSV')
        for frame in frame_list:
            changed_frame_list.append(cv2.cvtColor(frame, cv2.COLOR_BGR2HSV))
        return changed_frame_list
    if target_colorspace=='cie-lab':
        print('cie-lab')
        for frame in frame_list:
            changed_frame_list.append(cv2.cvtColor(frame, cv2.COLOR_BGR2LAB))
        return changed_frame_list
    else:
        print('rgb')
        for frame in frame_list:
            changed_frame_list.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        return changed_frame_list

In [ ]:
def fn_rgb_to_color(target_colorspace,path):
            if (path != 'full'):
                colors = {}
                with open(path) as f:
                    for line in f:
                        #split lines at "::
                        color, rgb = line.strip().split(':')
                        #strip the rgb-string of the parenthesis, split it up a the commas,
                        #cast them to int and put them into a tuples
                        rgb_value=tuple(map(int,(rgb.strip('(').strip(')').split(','))))
                        colors[color]=rgb_value
            else:
                colors={'darkred':(139,0,0),
                'firebrick':(178,34,34),
                'crimson':(220,20,60),
                'red':(255,0,0),
                'tomato':(255,99,71),
                'salmon':(250,128,114),
                'darkorange':(255,140,0),
                'gold':(255,215,0),
                'darkkhaki':(189,183,107),
                'yellow':(255,255,0),
                'darkolivegreen':(85,107,47),
                'olivedrab':(107,142,35),
                'greenyellow':(173,255,47),
                'darkgreen':(0,100,0),
                'aquamarine':(127,255,212),
                'steelblue':(70,130,180),
                'skyblue':(135,206,235),
                'darkblue':(0,0,139),
                'blue':(0,0,255),
                'royalblue':(65,105,225),
                'purple':(128,0,128),
                'violet':(238,130,238),
                'deeppink':(255,20,147),
                'pink':(255,192,203),
                'antiquewhite':(250,235,215),
                'saddlebrown':(139,69,19),
                'sandybrown':(244,164,96),
                'ivory':(255,255,240),
                'dimgrey':(105,105,105),
                'grey':(28,128,128),
                'silver':(192,192,192),
                'lightgrey':(211,211,211),
                'black':(0,0,0),
                'white':(255,255,255),
                'darkcyan':(0,139,139),
                'cyan':(0,255,255),
                'green':(0,128,0),
                'khaki':(240,230,140),
                'goldenrod':(218,165,32),
                'orange':(255,165,0),
                'coral':(255,127,80),
                'magenta':(255,0,255),
                'wheat':(245,222,179),
                'skin':(255,224,189),
                'purple4':(147,112,219)}

            colors_aux={}
            if target_colorspace=='HSV':
                print('HSV')
                for color in colors:
                    a = np.array((colors[color]),dtype='uint8')
                    b = a.reshape(1,1,3)
                    c = cv2.cvtColor(b,cv2.COLOR_RGB2HSV)
                    colors_aux[color]=tuple(c.reshape(3))
                colors=colors_aux
            if target_colorspace=='cie-lab':
                print('cie-lab')
                for color in colors:
                    a = np.array((colors[color]),dtype='uint8')
                    b = a.reshape(1,1,3)
                    c = cv2.cvtColor(b,cv2.COLOR_RGB2LAB)
                    colors_aux[color]=tuple(c.reshape(3))
                colors=colors_aux

            rgb_to_color={}
            for color in colors:
                rgb_to_color[colors[color]]=color
            #purple4 is median purple
            #skin is caucasian        
            return rgb_to_color

In [ ]:
#params
resolution=200

In [ ]:
scene_all_frames_lab=[]
for begin,end in zip(ts_begin,ts_end):
    scene_all_frames_lab.append(read_video_segments('../Wells_John_CompanyMen_full.mp4',begin,end,resolution,'cie-lab'))

In [ ]:
# frames_hsv=read_video_segments('../Wells_John_CompanyMen_full.mp4',begin[0],end[0],resolution,'HSV')

In [ ]:
def extract_dominant_colors(frame_list,target_colorspace,path,what_to_process):
    print(str(len(frame_list))+' frames to process.')
    rgb_to_color=fn_rgb_to_color(target_colorspace,path) #get the color dict 
    bins={} #bins dict for histograms 
    for rgb in rgb_to_color: #init the dict with zeros for every key
        bins[rgb_to_color[rgb]]=0
    rgb_list=[] #create a traverseable list of the rgb_values
    for rgb in rgb_to_color: #map the values of the dict to a list
        rgb_list.append(rgb)
    i = 0

    kdt = KDTree(rgb_list, leaf_size=30, metric='euclidean')
    if what_to_process=='scene':
        for frames in tqdm(frame_list): #traverse the video
            for image in frames:
                img = image.reshape((image.shape[0] * image.shape[1], 3)) #flatten the image to 1d   
                nns = kdt.query(img, k=1, return_distance=False)
                for nn in nns:
                    bins[rgb_to_color[rgb_list[nn[0]]]]+=1
                i+=1
    else:
        for image in tqdm(frame_list): #traverse the video
            img = image.reshape((image.shape[0] * image.shape[1], 3)) #flatten the image to 1d   
            nns = kdt.query(img, k=1, return_distance=False)
            for nn in nns:
                bins[rgb_to_color[rgb_list[nn[0]]]]+=1
            i+=1
    norm_factor = len(frame_list)* np.shape(frame_list[0])[0] * np.shape(frame_list[0])[1] #normalize the bins
    bins_norm={k:v/norm_factor for k,v in bins.items()}
    return bins_norm

In [ ]:
def bins_to_df(bins,bin_threshold=5,colors_to_return=5):
    #create a dataframe, sorted descending by count
    bins_sorted=sorted(zip(list(bins.values()),list(bins.keys())),reverse=True)
    df=pd.DataFrame(bins_sorted,columns=['count','color'])
    df.set_index('color',inplace=True) #set the colors as the index of the dataframe
#     bin_threshold=bin_threshold/100 #scale the percentage to 0-1
#     df = df[df>bin_threshold].dropna() #kick bins from the dataframe with precentage lower than bin_threshold 
    return df#.head(colors_to_return)#return the color_return highest bins, default 5, if less bins then
                                    #color_return are there return all

In [ ]:
hist=extract_dominant_colors(frames_lab,'cie-lab','full','shot')

In [ ]:
a=bins_to_df(hist)
b=bins_to_df(hist)
a-b

In [ ]:
# def dominant_test(frame_list,target_colorspace,path,what_to_process):
#     print(str(len(frame_list))+' frames to process.')
#     rgb_to_color=fn_rgb_to_color(target_colorspace,path) #get the color dict 
#     bin_list=[]
#     bins={} #bins dict for histograms 
#     for rgb in rgb_to_color: #init the dict with zeros for every key
#         bins[rgb_to_color[rgb]]=0
#     rgb_list=[] #create a traverseable list of the rgb_values
#     for rgb in rgb_to_color: #map the values of the dict to a list
#         rgb_list.append(rgb)
#     i = 0
#     kdt = KDTree(rgb_list, leaf_size=30, metric='euclidean')
    
#     for frames in tqdm(frame_list):
#         bin_aux=bins
#         for image in frames: #traverse the video
#             img = image.reshape((image.shape[0] * image.shape[1], 3)) #flatten the image to 1d   
#             nns = kdt.query(img, k=1, return_distance=False)
#             for nn in nns:
#                 bin_aux[rgb_to_color[rgb_list[nn[0]]]]+=1
#             i+=1
#     norm_factor = len(frame_list)* np.shape(frame_list[0])[0] * np.shape(frame_list[0])[1] #normalize the bins
#     bins_norm={k:v/norm_factor for k,v in bins.items()}
#     return bins_norm